# Q3: ARIMA - Quy Trình Ra Quyết Định

Notebook này giải thích **quy trình hoàn chỉnh** để xây dựng mô hình ARIMA cho dự báo PM2.5:

1. **Quan sát chuỗi gốc** - Nhận diện xu hướng và mùa vụ
2. **Kiểm định dừng** (ADF/KPSS) - Chọn d
3. **Xem ACF/PACF** - Đoán p và q
4. **Grid search** - Tìm (p,d,q) tối ưu
5. **Chẩn đoán phần dư** - Kiểm tra residuals

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## Load và Chuẩn bị Dữ liệu

In [ ]:
from src.classification_library import load_beijing_air_quality, clean_air_quality_df

RAW_ZIP_PATH = '../data/raw/PRSA2017_Data_20130301-20170228.zip'
STATION = 'Aotizhongxin'
VALUE_COL = 'PM2.5'
CUTOFF = '2017-01-01'

# Load data
df_raw = load_beijing_air_quality(use_ucimlrepo=False, raw_zip_path=RAW_ZIP_PATH)
df = clean_air_quality_df(df_raw)

# Lọc trạm và tạo time series
df_station = df[df['station'] == STATION].sort_values('datetime').reset_index(drop=True)
df_ts = df_station[['datetime', VALUE_COL]].dropna().copy()
df_ts = df_ts.set_index('datetime')

print(f"Time series shape: {df_ts.shape}")
print(f"Date range: {df_ts.index.min()} to {df_ts.index.max()}")
df_ts.head()

In [ ]:
# Split train/test
train = df_ts[df_ts.index < CUTOFF]
test = df_ts[df_ts.index >= CUTOFF]

print(f"Train: {len(train)} observations ({train.index.min()} to {train.index.max()})")
print(f"Test:  {len(test)} observations ({test.index.min()} to {test.index.max()})")

## BƯỚC 1: Quan sát Chuỗi Gốc - Nhận diện Xu hướng và Mùa vụ

**Mục tiêu:**
- Xem chuỗi có xu hướng (trend) không?
- Có tính mùa vụ (seasonality) không?
- Có pattern lặp lại không?

**Phương pháp:**
- Vẽ đồ thị toàn bộ chuỗi
- Tính moving average để làm mượt
- Decomposition (nếu cần)

In [ ]:
print("=" * 70)
print("BƯỚC 1: QUAN SÁT CHUỖI GỐC")
print("=" * 70)

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Plot 1: Original series
ax1 = axes[0]
ax1.plot(train.index, train[VALUE_COL], linewidth=0.8, alpha=0.7, color='steelblue')
ax1.set_ylabel(VALUE_COL, fontsize=11)
ax1.set_title(f'{VALUE_COL} Original Series (Training Data)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: With moving averages
ax2 = axes[1]
ax2.plot(train.index, train[VALUE_COL], linewidth=0.5, alpha=0.5, color='lightblue', label='Original')
train_copy = train.copy()
train_copy['MA_7d'] = train_copy[VALUE_COL].rolling(window=24*7, center=True).mean()
train_copy['MA_30d'] = train_copy[VALUE_COL].rolling(window=24*30, center=True).mean()
ax2.plot(train_copy.index, train_copy['MA_7d'], linewidth=2, color='orange', label='7-day MA')
ax2.plot(train_copy.index, train_copy['MA_30d'], linewidth=2, color='red', label='30-day MA')
ax2.set_ylabel(VALUE_COL, fontsize=11)
ax2.set_title('Với Moving Averages (để quan sát xu hướng)', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Monthly aggregation
ax3 = axes[2]
monthly_mean = train.resample('ME')[VALUE_COL].mean()
ax3.plot(monthly_mean.index, monthly_mean.values, marker='o', linewidth=2, markersize=6, color='green')
ax3.set_ylabel(f'Monthly Mean {VALUE_COL}', fontsize=11)
ax3.set_xlabel('Time', fontsize=11)
ax3.set_title('Trung bình theo tháng (để quan sát seasonality)', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 QUAN SÁT:")
print("- Có xu hướng dài hạn (tăng/giảm) không?")
print("- Có pattern lặp lại theo mùa không?")
print("- Phương sai có thay đổi theo thời gian không?")
print("\n→ Quan sát này giúp quyết định có cần differencing và seasonality component không")

## BƯỚC 2: Kiểm định Dừng (Stationarity) - Chọn d

**Mục tiêu:** Xác định tham số **d** (order of differencing)

**Phương pháp:**
1. ADF test (Augmented Dickey-Fuller)
   - H0: chuỗi không dừng (có unit root)
   - p < 0.05 → reject H0 → chuỗi dừng

2. KPSS test
   - H0: chuỗi dừng
   - p < 0.05 → reject H0 → chuỗi không dừng

**Quy tắc chọn d:**
- Chuỗi dừng → d=0
- Chuỗi không dừng → thử d=1 (first difference)
- Nếu vẫn chưa dừng → thử d=2

In [ ]:
print("=" * 70)
print("BƯỚC 2: KIỂM ĐỊNH DỪNG - CHỌN d")
print("=" * 70)

def test_stationarity(series, series_name="Series"):
    """Kiểm định tính dừng bằng ADF và KPSS"""
    print(f"\n{series_name}:")
    print("-" * 60)
    
    # ADF Test
    adf_result = adfuller(series.dropna(), autolag='AIC')
    print(f"ADF Test:")
    print(f"  Statistic: {adf_result[0]:.4f}")
    print(f"  P-value:   {adf_result[1]:.4f}")
    print(f"  Critical values: {adf_result[4]}")
    
    adf_stationary = adf_result[1] < 0.05
    print(f"  → {'✅ DỪNG' if adf_stationary else '❌ KHÔNG DỪNG'} (ADF)")
    
    # KPSS Test
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')
    print(f"\nKPSS Test:")
    print(f"  Statistic: {kpss_result[0]:.4f}")
    print(f"  P-value:   {kpss_result[1]:.4f}")
    print(f"  Critical values: {kpss_result[3]}")
    
    kpss_stationary = kpss_result[1] >= 0.05
    print(f"  → {'✅ DỪNG' if kpss_stationary else '❌ KHÔNG DỪNG'} (KPSS)")
    
    # Kết luận
    if adf_stationary and kpss_stationary:
        conclusion = "✅✅ CẢ HAI TEST ĐỒNG Ý: DỪNG"
        is_stationary = True
    elif not adf_stationary and not kpss_stationary:
        conclusion = "❌❌ CẢ HAI TEST ĐỒNG Ý: KHÔNG DỪNG"
        is_stationary = False
    else:
        conclusion = "⚠️ HAI TEST MÂU THUẪN - cần xem xét thêm"
        is_stationary = False
    
    print(f"\n{conclusion}")
    return is_stationary

# Test original series
series = train[VALUE_COL]
is_stationary_d0 = test_stationarity(series, "Chuỗi GỐC (d=0)")

In [ ]:
# Nếu không dừng, thử differencing
if not is_stationary_d0:
    print("\n" + "=" * 70)
    print("Chuỗi gốc KHÔNG DỪNG → Thử DIFFERENCING")
    print("=" * 70)
    
    # First difference
    series_diff1 = series.diff().dropna()
    is_stationary_d1 = test_stationarity(series_diff1, "First Difference (d=1)")
    
    if not is_stationary_d1:
        # Second difference
        series_diff2 = series_diff1.diff().dropna()
        is_stationary_d2 = test_stationarity(series_diff2, "Second Difference (d=2)")
        
        if is_stationary_d2:
            recommended_d = 2
        else:
            recommended_d = 1  # fallback
    else:
        recommended_d = 1
else:
    recommended_d = 0

print("\n" + "=" * 70)
print(f"📌 KHUYẾN NGHỊ: d = {recommended_d}")
print("=" * 70)

In [ ]:
# Visualize differencing effect
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Original
axes[0].plot(series.values[:1000], linewidth=1, color='blue')
axes[0].set_title('Chuỗi Gốc (d=0)', fontsize=12, fontweight='bold')
axes[0].set_ylabel(VALUE_COL)
axes[0].grid(True, alpha=0.3)

# First difference
if recommended_d >= 1:
    axes[1].plot(series_diff1.values[:1000], linewidth=1, color='orange')
    axes[1].set_title('First Difference (d=1)', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Δ ' + VALUE_COL)
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[1].grid(True, alpha=0.3)

# Second difference
if recommended_d >= 2:
    axes[2].plot(series_diff2.values[:1000], linewidth=1, color='green')
    axes[2].set_title('Second Difference (d=2)', fontsize=12, fontweight='bold')
    axes[2].set_ylabel('Δ² ' + VALUE_COL)
    axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## BƯỚC 3: Xem ACF/PACF - Đoán p và q

**Mục tiêu:** Xác định tham số **p** (AR order) và **q** (MA order)

**Quy tắc đơn giản:**

| Pattern | ACF | PACF | Model |
|---------|-----|------|-------|
| AR(p) | Decay dần | Cut off sau lag p | p = số lag trước khi PACF cut off |
| MA(q) | Cut off sau lag q | Decay dần | q = số lag trước khi ACF cut off |
| ARMA(p,q) | Decay dần | Decay dần | Cần thử nhiều giá trị |

**Thực tế:**
- Thường khó xác định chính xác từ ACF/PACF
- Dùng ACF/PACF để gợi ý khoảng giá trị (ví dụ: p ≤ 3, q ≤ 3)
- Sau đó dùng grid search để tìm giá trị tối ưu

In [ ]:
print("=" * 70)
print("BƯỚC 3: XEM ACF/PACF - ĐOÁN p và q")
print("=" * 70)

# Chọn chuỗi phù hợp (sau differencing nếu cần)
if recommended_d == 0:
    series_for_acf = series
elif recommended_d == 1:
    series_for_acf = series_diff1
else:
    series_for_acf = series_diff2

# Plot ACF and PACF
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# ACF
plot_acf(series_for_acf.dropna(), lags=40, ax=axes[0])
axes[0].set_title(f'ACF - Autocorrelation Function (d={recommended_d})', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=11)

# PACF
plot_pacf(series_for_acf.dropna(), lags=40, ax=axes[1])
axes[1].set_title(f'PACF - Partial Autocorrelation Function (d={recommended_d})', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=11)

plt.tight_layout()
plt.show()

print("\n📊 HƯỚNG DẪN ĐỌC:")
print("-" * 60)
print("ACF (Autocorrelation):")
print("  - Nếu cut off đột ngột sau lag q → model có thành phần MA(q)")
print("  - Nếu decay dần → model có thành phần AR")
print("\nPACF (Partial Autocorrelation):")
print("  - Nếu cut off đột ngột sau lag p → model có thành phần AR(p)")
print("  - Nếu decay dần → model có thành phần MA")
print("\n⚠️ Trong thực tế, pattern thường không rõ ràng như sách vở")
print("→ Dùng ACF/PACF để gợi ý khoảng giá trị, sau đó dùng grid search")

In [ ]:
# Gợi ý p và q từ ACF/PACF
acf_values = acf(series_for_acf.dropna(), nlags=20)
pacf_values = pacf(series_for_acf.dropna(), nlags=20)

# Tìm lag đầu tiên mà ACF/PACF không significant (trong CI)
# CI ≈ ±1.96/sqrt(n) for 95% confidence
n = len(series_for_acf.dropna())
ci = 1.96 / np.sqrt(n)

# Tìm q candidate (từ ACF)
q_candidates = []
for i in range(1, len(acf_values)):
    if abs(acf_values[i]) > ci:
        q_candidates.append(i)
    elif len(q_candidates) > 0:
        break  # stop at first insignificant lag after significant ones

# Tìm p candidate (từ PACF)
p_candidates = []
for i in range(1, len(pacf_values)):
    if abs(pacf_values[i]) > ci:
        p_candidates.append(i)
    elif len(p_candidates) > 0:
        break

suggested_p_max = max(p_candidates) if p_candidates else 1
suggested_q_max = max(q_candidates) if q_candidates else 1

# Giới hạn để tránh quá phức tạp
suggested_p_max = min(suggested_p_max, 5)
suggested_q_max = min(suggested_q_max, 5)

print("\n📌 GỢI Ý TỪ ACF/PACF:")
print("=" * 70)
print(f"p (AR order):  thử từ 0 đến {suggested_p_max}")
print(f"q (MA order):  thử từ 0 đến {suggested_q_max}")
print(f"d (differencing): {recommended_d}")
print("\n→ Sẽ dùng grid search để tìm (p,d,q) tối ưu trong khoảng này")

## BƯỚC 4: Grid Search - Tìm (p,d,q) tối ưu

**Mục tiêu:** Tìm tổ hợp (p,d,q) cho AIC/BIC thấp nhất

**AIC vs BIC:**
- AIC (Akaike Information Criterion): cân bằng giữa goodness-of-fit và độ phức tạp
- BIC (Bayesian Information Criterion): phạt mô hình phức tạp nặng hơn AIC
- Thường dùng AIC cho forecasting, BIC cho model selection

**Quy tắc:**
- Càng thấp càng tốt
- Ưu tiên model đơn giản nếu AIC/BIC gần nhau

In [ ]:
print("=" * 70)
print("BƯỚC 4: GRID SEARCH - TÌM (p,d,q) TỐI ƯU")
print("=" * 70)

# Grid search
p_range = range(0, suggested_p_max + 1)
d_range = [recommended_d]  # Giữ d cố định
q_range = range(0, suggested_q_max + 1)

results = []

print(f"\nThử {len(p_range)} x {len(d_range)} x {len(q_range)} = {len(p_range)*len(d_range)*len(q_range)} models...")
print("(Có thể mất vài phút)\n")

for p in p_range:
    for d in d_range:
        for q in q_range:
            try:
                model = ARIMA(train[VALUE_COL], order=(p, d, q))
                fitted = model.fit()
                results.append({
                    'p': p, 'd': d, 'q': q,
                    'aic': fitted.aic,
                    'bic': fitted.bic,
                    'order': (p, d, q)
                })
            except:
                continue

results_df = pd.DataFrame(results)

# Top 10 by AIC
print("\nTOP 10 MODELS BY AIC:")
print("=" * 70)
top_aic = results_df.nsmallest(10, 'aic')
print(top_aic.to_string(index=False))

# Best model
best_by_aic = results_df.loc[results_df['aic'].idxmin()]
best_order_aic = (int(best_by_aic['p']), int(best_by_aic['d']), int(best_by_aic['q']))

print("\n" + "=" * 70)
print(f"📌 BEST MODEL BY AIC: ARIMA{best_order_aic}")
print(f"   AIC = {best_by_aic['aic']:.2f}")
print(f"   BIC = {best_by_aic['bic']:.2f}")
print("=" * 70)

In [ ]:
# Visualize grid search results
if len(results_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Heatmap for AIC
    pivot_aic = results_df.pivot_table(values='aic', index='p', columns='q', aggfunc='min')
    sns.heatmap(pivot_aic, annot=True, fmt='.0f', cmap='YlOrRd', ax=axes[0])
    axes[0].set_title(f'AIC Heatmap (d={recommended_d})', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('q (MA order)')
    axes[0].set_ylabel('p (AR order)')
    
    # Heatmap for BIC
    pivot_bic = results_df.pivot_table(values='bic', index='p', columns='q', aggfunc='min')
    sns.heatmap(pivot_bic, annot=True, fmt='.0f', cmap='YlGnBu', ax=axes[1])
    axes[1].set_title(f'BIC Heatmap (d={recommended_d})', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('q (MA order)')
    axes[1].set_ylabel('p (AR order)')
    
    plt.tight_layout()
    plt.show()

## BƯỚC 5: Chẩn đoán Phần dư (Residual Diagnostics)

**Mục tiêu:** Kiểm tra xem model đã bắt được cấu trúc chính của chuỗi chưa

**Phần dư TỐT phải:**
1. Có mean ≈ 0
2. Phân phối gần Normal
3. Không có autocorrelation (≈ white noise)
4. Phương sai ổn định (homoscedastic)

**Cách kiểm tra:**
- Plot residuals
- Histogram & Q-Q plot
- ACF of residuals
- Ljung-Box test (H0: no autocorrelation)

In [ ]:
print("=" * 70)
print(f"BƯỚC 5: CHẨN ĐOÁN PHẦN DƯ - ARIMA{best_order_aic}")
print("=" * 70)

# Fit best model
best_model = ARIMA(train[VALUE_COL], order=best_order_aic)
best_fitted = best_model.fit()

# Get residuals
residuals = best_fitted.resid

print(f"\nMODEL SUMMARY:")
print(best_fitted.summary())

In [ ]:
# Residual diagnostics plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Residuals over time
ax1 = axes[0, 0]
ax1.plot(residuals, linewidth=0.8, color='steelblue')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax1.set_title('Residuals Over Time', fontsize=12, fontweight='bold')
ax1.set_ylabel('Residuals')
ax1.grid(True, alpha=0.3)

# Plot 2: Histogram
ax2 = axes[0, 1]
ax2.hist(residuals, bins=50, density=True, alpha=0.7, color='coral', edgecolor='black')
# Overlay normal distribution
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
from scipy.stats import norm
ax2.plot(x, norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal fit')
ax2.set_title('Residuals Distribution', fontsize=12, fontweight='bold')
ax2.set_xlabel('Residuals')
ax2.set_ylabel('Density')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Q-Q plot
from scipy import stats
ax3 = axes[1, 0]
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: ACF of residuals
ax4 = axes[1, 1]
plot_acf(residuals, lags=40, ax=ax4)
ax4.set_title('ACF of Residuals', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Statistical tests on residuals
print("\n" + "=" * 70)
print("KIỂM ĐỊNH THỐNG KÊ CHO PHẦN DƯ")
print("=" * 70)

# 1. Mean test
print(f"\n1. Mean of residuals: {residuals.mean():.6f}")
if abs(residuals.mean()) < 0.1 * residuals.std():
    print("   ✅ Mean ≈ 0 (good)")
else:
    print("   ⚠️ Mean khác 0 đáng kể")

# 2. Normality test (Jarque-Bera)
from scipy.stats import jarque_bera
jb_stat, jb_pvalue = jarque_bera(residuals)
print(f"\n2. Jarque-Bera test (normality):")
print(f"   Statistic: {jb_stat:.4f}")
print(f"   P-value: {jb_pvalue:.6f}")
if jb_pvalue > 0.05:
    print("   ✅ Residuals gần phân phối Normal (p > 0.05)")
else:
    print("   ⚠️ Residuals không phân phối Normal (p < 0.05)")

# 3. Ljung-Box test (autocorrelation)
lb_test = acorr_ljungbox(residuals, lags=[10, 20, 30], return_df=True)
print(f"\n3. Ljung-Box test (no autocorrelation):")
print(lb_test)
if (lb_test['lb_pvalue'] > 0.05).all():
    print("   ✅ Không có autocorrelation đáng kể (all p > 0.05)")
    print("   → Residuals ≈ white noise (GOOD!)")
else:
    print("   ⚠️ Vẫn còn autocorrelation (some p < 0.05)")
    print("   → Model chưa bắt hết cấu trúc, có thể cần model phức tạp hơn")

# 4. Heteroscedasticity (visual check)
print(f"\n4. Heteroscedasticity (visual):")
print("   → Xem Plot 1: Residuals Over Time")
print("   → Nếu phương sai không đổi theo thời gian = homoscedastic (good)")
print("   → Nếu phương sai thay đổi theo thời gian = heteroscedastic (bad)")

In [ ]:
# Final assessment
print("\n" + "=" * 70)
print("📊 TỔNG KẾT CHẨN ĐOÁN")
print("=" * 70)

checks = {
    'Mean ≈ 0': abs(residuals.mean()) < 0.1 * residuals.std(),
    'Normal distribution': jb_pvalue > 0.05,
    'No autocorrelation': (lb_test['lb_pvalue'] > 0.05).all(),
}

passed = sum(checks.values())
total = len(checks)

for check, result in checks.items():
    print(f"{'✅' if result else '❌'} {check}")

print(f"\nPassed: {passed}/{total}")

if passed == total:
    print("\n🎉 XUẤT SẮC! Model đã bắt được cấu trúc chính của chuỗi")
    print("   → Residuals ≈ white noise")
    print("   → Model sẵn sàng để forecast")
elif passed >= total - 1:
    print("\n✅ TỐT! Model tương đối tốt")
    print("   → Có thể dùng để forecast, nhưng có thể cải thiện")
else:
    print("\n⚠️ CẦN CẢI THIỆN! Model chưa tối ưu")
    print("   → Residuals vẫn có structure")
    print("   → Xem xét:")
    print("      • Thử (p,d,q) khác")
    print("      • Thêm seasonal component (SARIMA)")
    print("      • Xem xét model khác (GARCH cho volatility, etc.)")

## BONUS: Forecast với Model Tốt Nhất

In [ ]:
# Forecast
n_forecast = len(test)
forecast = best_fitted.forecast(steps=n_forecast)

# Calculate metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(test[VALUE_COL], forecast))
mae = mean_absolute_error(test[VALUE_COL], forecast)

print("\n" + "=" * 70)
print("FORECAST PERFORMANCE")
print("=" * 70)
print(f"Model: ARIMA{best_order_aic}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

# Plot train (last 1000 points)
train_plot = train[VALUE_COL].iloc[-1000:]
ax.plot(train_plot.index, train_plot.values, label='Train (last 1000)', linewidth=1, color='blue')

# Plot test
ax.plot(test.index, test[VALUE_COL].values, label='Actual', linewidth=1.5, color='green')

# Plot forecast
ax.plot(test.index, forecast.values, label=f'Forecast ARIMA{best_order_aic}', 
        linewidth=1.5, color='red', linestyle='--')

ax.axvline(x=test.index[0], color='black', linestyle='--', linewidth=2, alpha=0.5, label='Train/Test split')
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel(VALUE_COL, fontsize=11)
ax.set_title(f'Forecast vs Actual - ARIMA{best_order_aic}', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📝 TÓM TẮT QUY TRÌNH ARIMA

### ✅ 5 BƯỚC RA QUYẾT ĐỊNH:

**BƯỚC 1: QUAN SÁT CHUỖI GỐC**
- Vẽ đồ thị, thêm moving average
- Nhận diện xu hướng (trend) và mùa vụ (seasonality)
- Quyết định: Có cần seasonal component không?

**BƯỚC 2: KIỂM ĐỊNH DỪNG - CHỌN d**
- Chạy ADF test và KPSS test
- Nếu không dừng → differencing
- Quyết định: d = 0, 1, hoặc 2

**BƯỚC 3: XEM ACF/PACF - GỢI Ý p và q**
- Plot ACF và PACF của chuỗi sau differencing
- Tìm pattern cut-off hoặc decay
- Quyết định: Khoảng giá trị cho p và q (ví dụ: p ≤ 3, q ≤ 3)

**BƯỚC 4: GRID SEARCH - TÌM (p,d,q) TỐI ƯU**
- Thử tất cả combinations trong khoảng
- So sánh AIC/BIC
- Quyết định: (p,d,q) có AIC thấp nhất

**BƯỚC 5: CHẨN ĐOÁN PHẦN DƯ**
- Kiểm tra residuals: mean, normality, autocorrelation
- Ljung-Box test
- Quyết định: Model đủ tốt? Hay cần thử (p,d,q) khác?

### ⚠️ ĐIỂM QUAN TRỌNG:

1. **Residuals ≈ White Noise là mục tiêu**
   - Mean ≈ 0
   - No autocorrelation
   - Normal distribution (tốt nhưng không bắt buộc)

2. **AIC/BIC chỉ là công cụ**
   - Không phải càng thấp càng tốt mọi lúc
   - Ưu tiên model đơn giản nếu performance gần nhau

3. **Residual diagnostics là QUAN TRỌNG NHẤT**
   - Model với AIC thấp nhất có thể không tốt nếu residuals kém
   - Kiểm tra kỹ trước khi dùng forecast

### 💡 MỞ RỘNG:

Nếu ARIMA không đủ tốt, xem xét:
- **SARIMA**: Thêm seasonal component (P,D,Q,s)
- **ARIMAX**: Thêm external regressors (biến ngoại sinh)
- **GARCH**: Model cho volatility clustering
- **Prophet, LSTM**: Các phương pháp hiện đại hơn